# Dementia Prediction Using the Neural Network Model

In [1]:
# Import modules
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
from sqlalchemy import create_engine
import tensorflow as tf

## Load Data from SQLite Database

In [2]:
# Create SQLAlchemy engine for SQLite
engine = create_engine('sqlite:///Data/dementia_data.db')

# Retrieve data from SQLite Database
query = "SELECT * FROM dementia_data"
df = pd.read_sql(query, con=engine)

# Preview DataFrame
df.head()

,Diabetic,AlcoholLevel,HeartRate,BloodOxygenLevel,BodyTemperature,Weight,MRI_Delay,Prescription,Dosage in mg,Age,...,Smoking_Status,APOE_ε4,Physical_Activity,Depression_Status,Cognitive_Test_Scores,Medication_History,Nutrition_Diet,Sleep_Quality,Chronic_Health_Conditions,Dementia
0,1,0.084974,98,96.230743,36.224852,57.563978,36.421028,None,NaN,60,...,Current Smoker,Negative,Sedentary,No,10,No,Low-Carb Diet,Poor,Diabetes,0
1,0,0.016973,78,93.032122,36.183874,56.832335,31.157633,Galantamine,12.0,61,...,Former Smoker,Positive,Moderate Activity,No,1,Yes,Low-Carb Diet,Poor,Heart Disease,1
2,0,0.009000,89,93.566504,37.326321,59.759066,37.640435,None,NaN,69,...,Former Smoker,Negative,Moderate Activity,No,8,No,Mediterranean Diet,Poor,Heart Disease,0
3,0,0.086437,60,93.906510,37.030620,58.266471,50.673992,Donepezil,23.0,78,...,Never Smoked,Negative,Mild Activity,Yes,5,Yes,Balanced Diet,Poor,Hypertension,1
4,1,0.150747,67,97.508994,36.062121,67.705027,27.810601,Memantine,20.0,77,...,Never Smoked,Positive,Mild Activity,No,0,Yes,Low-Carb Diet,Good,Diabetes,1


In [3]:
# Impute 'Dosage in mg' with zero
df['Dosage in mg'] = df['Dosage in mg'].fillna(0)

# Fill missing 'Prescription' values with 'None'
df['Prescription'].fillna('None', inplace=True)

# Preview DataFrame
df.head()

,Diabetic,AlcoholLevel,HeartRate,BloodOxygenLevel,BodyTemperature,Weight,MRI_Delay,Prescription,Dosage in mg,Age,...,Smoking_Status,APOE_ε4,Physical_Activity,Depression_Status,Cognitive_Test_Scores,Medication_History,Nutrition_Diet,Sleep_Quality,Chronic_Health_Conditions,Dementia
0,1,0.084974,98,96.230743,36.224852,57.563978,36.421028,None,0.0,60,...,Current Smoker,Negative,Sedentary,No,10,No,Low-Carb Diet,Poor,Diabetes,0
1,0,0.016973,78,93.032122,36.183874,56.832335,31.157633,Galantamine,12.0,61,...,Former Smoker,Positive,Moderate Activity,No,1,Yes,Low-Carb Diet,Poor,Heart Disease,1
2,0,0.009000,89,93.566504,37.326321,59.759066,37.640435,None,0.0,69,...,Former Smoker,Negative,Moderate Activity,No,8,No,Mediterranean Diet,Poor,Heart Disease,0
3,0,0.086437,60,93.906510,37.030620,58.266471,50.673992,Donepezil,23.0,78,...,Never Smoked,Negative,Mild Activity,Yes,5,Yes,Balanced Diet,Poor,Hypertension,1
4,1,0.150747,67,97.508994,36.062121,67.705027,27.810601,Memantine,20.0,77,...,Never Smoked,Positive,Mild Activity,No,0,Yes,Low-Carb Diet,Good,Diabetes,1


# Base Model (No columns dropped)

## Preprocessing

In [4]:
# Convert categorical data to numeric
base_df = pd.get_dummies(df, dtype="int")

In [5]:
# Split data into features and target arrays
# Target
y = base_df["Dementia"]

# Features
X = base_df.drop(columns = "Dementia")

# Split into a training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train & Evaluate Model

In [7]:
# Define the neural network model
nn = tf.keras.models.Sequential()

# Input Layer
nn.add(tf.keras.layers.InputLayer(shape=(45,)))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           460 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 471 (1.84 KB)

 Trainable params: 471 (1.84 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - accuracy: 0.5436 - loss: 1.7570
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.6278 - loss: 1.0427
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 636us/step - accuracy: 0.6367 - loss: 0.9616
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.7103 - loss: 0.6793
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 615us/step - accuracy: 0.7679 - loss: 0.5570
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - accuracy: 0.8294 - loss: 0.3997
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.8452 - loss: 0.3524
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step - accuracy: 0.8725 - loss: 0.3063
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.9038 - loss: 0.2314
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - accuracy: 0.9148 - loss: 0.2272
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.9457 - loss: 0.1596
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)

8/8 - 0s - 11ms/step - accuracy: 0.9960 - loss: 0.0172
